# MovieLen data preparation

For reference!

This is how I prepared the data for Homework 2.

## Import LensKit

Has good tools for data handling

In [10]:
import sys
!{sys.executable} -m pip install lenskit

In [ ]:
from numpy import random
from lenskit.datasets import MovieLens
import lenskit.crossfold as xf

## Load the data

In [11]:
ds = MovieLens('~/Downloads/ml-25m')

## Choose a small subset

Sample approx. 2.5% of the users (4000) just to make the homework tractable

In [12]:
users = ds.ratings['user'].unique()

In [16]:
len(users)

162541

In [18]:
users_4k = random.choice(users, 4000)

In [21]:
ratings_4k = ds.ratings[ds.ratings['user'].isin(users_4k)]

In [29]:
ratings_4k.describe()

,user,item,rating,timestamp
count,609165.000000,609165.000000,609165.000000,6.091650e+05
mean,82896.185630,20697.145033,3.555168,1.211100e+09
std,46934.272073,38477.642446,1.068208,2.258935e+08
min,12.000000,1.000000,0.500000,8.228736e+08
25%,42992.000000,1197.000000,3.000000,1.008047e+09
50%,82570.000000,2918.000000,4.000000,1.186240e+09
75%,123534.000000,8254.000000,4.000000,1.441574e+09
max,162499.000000,208741.000000,5.000000,1.574310e+09


### Drop items with frequency less than 10

In [32]:
count_table = ratings_4k['item'].value_counts()

In [45]:
count_table_10 = count_table[count_table > 10]

In [46]:
ratings_4k_2 = ratings_4k[ratings_4k['item'].isin(count_table_10.index)]

In [49]:
ratings_4k_2['user'].value_counts()

100260    2215
129695    2033
122236    1949
75582     1773
132754    1772
          ... 
105801      13
132887      10
18247       10
59669       10
109980       3
Name: user, Length: 3957, dtype: int64

There's one user with only three ratings now. Drop that one.

In [51]:
ratings_4k_final = ratings_4k_2[ratings_4k_2['user']!=109980]

## Create train/test partitions of the data

The `partition_users` method says we are splitting user profiles. The `SampleN` method ensures that there are a fixed # of test items per user.

In [55]:
testRowsPerUsers = 5
for i, tp in enumerate(xf.partition_users(ratings_4k_final, 5, xf.SampleN(testRowsPerUsers)), 1):
    #_log.info('writing train set %d', i)
    tp.train.to_csv(f'train-{i}.csv', index=False)
    #_log.info('writing test set %d', i)
    tp.test.to_csv(f'test-{i}.csv', index=False)

## Get the relevant tag information

The tags only for the movies that are in the (filtered) dataset.

In [61]:
my_movies = ratings_4k_final['item'].unique()

In [59]:
import pandas as pd

tags_df = pd.read_csv('~/Downloads/ml-25m/genome-scores.csv')

In [62]:
tags_selected = tags_df[tags_df['movieId'].isin(my_movies)]

In [65]:
len(tags_selected['movieId'].unique())

5709

In [67]:
tags_selected.to_csv('sample-genome-scores.csv', index=False)